# Part 5 - Statistics

This section aims to calculate metrics described in the paper using our dataset.

### Imports, Declarations & Setup

In [ ]:
from matplotlib import pyplot as plt
from scipy import stats
import numpy as np
import csv
import json
import random
import itertools

random.seed("seeeeeeeed")

RECIPE_PATH = "data/no-overlap/"
RUSSIAN_RECIPE_PATHS = [RECIPE_PATH + "noOverlapRecipes.csv"]
WORLD_RECIPE_PATHS = [RECIPE_PATH + "paperNoOverlapRecipes.csv"]

CATEGORY_MAP_PATH = "data/subCategoryMap.json"
INGREDIENT_MAP_PATH = "data/notebook-3-data/ingredientMap (preprocessed_to_foodb).json"
COMPOUNDS_PATH = "data/compounds/compounds.json"

##### Load Dataset

In [ ]:
#load ingredient map
ingredientMap = {}
with open(INGREDIENT_MAP_PATH) as mapFile:
    ingredientMap = json.load(mapFile)

In [ ]:
#load category map
categoryMap = {}
with open(CATEGORY_MAP_PATH) as catMapFile:
    categoryMap = json.load(catMapFile)

In [ ]:
def loadRecipes(cuisine, recipe_paths):
    print("Loading: ", cuisine)
    #load recipes
    recipes = []
    categoryRecipes = []
    categoryComponents = dict()
    ingrFreq = dict()
    categoryFreq = dict()
    totalIngredientsUsed = 0

    for path in recipe_paths:
        with open(path) as file:
            count = 0
            reader = csv.reader(file)
            for row in reader:
                if row[0] not in cuisine:
                    continue
                count += 1
                # Get Recipes
                recipes.append({"name": row[0], "ingredients": row[1:]})
                for ingredient in row[1:]:
                    totalIngredientsUsed += 1
                    # Get Ingredient List and Ingredient Frequency
                    if ingredient not in ingrFreq:
                        ingrFreq[ingredient] = 1
                    else:
                        ingrFreq[ingredient] += 1
                    # Get Category List and Category Frequency
                    category = categoryMap[ingredient]
                    if category not in categoryFreq:
                        categoryFreq[category] = 1
                    else:
                        categoryFreq[category] += 1
                    # Construct Category to Category Components Mapping
                    if category not in categoryComponents:
                        categoryComponents[category] = set([ingredient])
                    else:
                        categoryComponents[category].add(ingredient)
                # Get Categories Recipe
                categoryRecipe = [categoryMap[ingredient] for ingredient in row[1:]]
                categoryRecipes.append({"name": row[0], "ingredients": categoryRecipe})
            print("Loaded {} recipes (\"{}\")".format(count, path))
    print("{} total recipes\n".format(len(recipes)))

    print("Sample Recipes:")
    print(recipes[0])
    print(recipes[-1])
    print()
    print()
    
    return recipes, categoryRecipes, categoryComponents, \
        ingrFreq, categoryFreq, totalIngredientsUsed

# Load Recipes
world_cuisine_list = ["NorthAmerican", "WesternEuropean", "LatinAmerican", "SouthernEuropean", "EastAsian"]
cuisine_recipes = {cuisine_name: loadRecipes([cuisine_name], WORLD_RECIPE_PATHS) for cuisine_name in world_cuisine_list}
cuisine_recipes["Russian"] = loadRecipes(["EasternEuropean","Russian"], RUSSIAN_RECIPE_PATHS)

### Number of Ingredients

In [ ]:
def numIngredients(figure_title, recipes):
    numIngredients = [len(recipe["ingredients"]) for recipe in recipes]

    ingredientCountMean = stats.tmean(numIngredients)
    ingredientCountStdDev = stats.tstd(numIngredients)

    skewness,loc,scale = stats.skewnorm.fit(numIngredients)
    x = np.linspace(stats.skewnorm.ppf(0.001, skewness, loc=loc, scale=scale), stats.skewnorm.ppf(0.999, skewness, loc=loc, scale=scale), 100)
    plt.plot(x, len(recipes)*stats.skewnorm.pdf(x, skewness, loc=loc, scale=scale), c="#333333", ls="--", lw=2, label='skewnormal approximation')

    plt.title(figure_title)
    plt.xlabel("Ingredient Count")
    plt.ylabel("Occurances")
    plt.hist(numIngredients, bins=max(numIngredients), color="#2e8b57")
    plt.vlines(ingredientCountMean, 0, plt.gca().get_ylim()[1], colors="r", label="Mean")
    plt.legend()
    plt.show()

    print("mean={}".format(ingredientCountMean))
    print("stddev={}".format(ingredientCountStdDev))
    print("range=({}, {})".format(min(numIngredients), max(numIngredients)))
    print("skew-normal distribution= (alpha={}, loc={}, scale={})\n".format(skewness,loc,scale))
    
    return skewness,loc,scale

# data[0] contains Recipes
figure_title = "Ingredient Distribution of %s Cuisine:"
ingredient_distribution = [numIngredients(figure_title % (cuisine_name),data[0])
                           for cuisine_name, data in cuisine_recipes.items()]

### Generate Random Recipe Set
These are used to determine the statistical significance of a cuisines mean number of shared recipe components $\Delta N_s$.

We are using four *null models* (methods of randomly generating recipes):

- **Uniform Random**: All ingredients are equally likely to be chosen. Common ingredients will tend to be chosen less than in real datasets and rare ingredients more often.
- **Frequency Conserving**: Ingredients will be randomly chosen with frequencies that approximate real ingredient usage.
- **Uniform Random + Category Preservation**: Ingredients will be chosen randomly but within a specific category; for example a recipe may have one random meat and three random vegetable ingredients.
- **Frequency Conserving + Category Preservation**: Like the above, ingredient categories are preserved but ingredients within those categories are chosen with frequencies matching their real distribution.

In [ ]:
INGREDIENTS_PATH = "../../data/ingredient-mapping/ingredients.json"
RANDOM_RECIPE_COUNT = 1000
   
# Define Function to Determine Number of Recipe's Component Ingredients
def drawNumIngredients(skewness,loc,scale):
    return max(1, int(stats.skewnorm.rvs(skewness, loc=loc, scale=scale)))

def generateRandomRecipes(recipes, categoryRecipes, categoryComponents, ingrFreq,
                          categoryFreq, totalIngredientsUsed, skewness, loc, scale):
    ## Get Ingredient List, Ingredient Frequency Proportion
    categoryIngredients = dict()
    categoryIngredientsProportion = dict()
    ingredients = []
    ingredientProportion = []
    for ingr, freq in ingrFreq.items():
        ingredients.append(ingr)
        ingredientProportion.append(freq/totalIngredientsUsed)
        category = categoryMap[ingr]
        if category not in categoryIngredients:
            categoryIngredients[category] = [ingr]
            categoryIngredientsProportion[category] = [freq/categoryFreq[category]]
        else:
            categoryIngredients[category].append(ingr)
            categoryIngredientsProportion[category].append(freq/categoryFreq[category])

    ## Generate Random Recipe Sets as Null Model

    # Ingredient Frequency Conserving
    randomRecipesWeighted = [{"name":"random{:04d}".format(num), 
                      "ingredients": np.random.choice(ingredients,size=drawNumIngredients(skewness,loc,scale), 
                                                      replace=False, p=ingredientProportion)
                     } for num in range(RANDOM_RECIPE_COUNT)]

    # Ingredient Frequency and Ingredient Category Preserving
    randomRecipesWeightedCategories = [{"name": recipe["name"],
                      "ingredients": [np.random.choice(categoryIngredients[categoryMap[ingredient]],
                                                       size=None,replace=False,
                                                       p=categoryIngredientsProportion[categoryMap[ingredient]])
                                for ingredient in recipe["ingredients"]]
                     } for recipe in recipes]

    # Uniform Random
    randomRecipes = [{"name":"random{:04d}".format(num), 
                      "ingredients": np.random.choice(ingredients,size=drawNumIngredients(skewness,loc,scale), 
                                                      replace=False)
                     } for num in range(RANDOM_RECIPE_COUNT)]

    # Uniform Random, Ingredient Category Preserving
    randomRecipesCategories = [{"name": recipe["name"],
                      "ingredients": [np.random.choice(categoryIngredients[categoryMap[ingredient]],
                                                       size=None,replace=False)
                                for ingredient in recipe["ingredients"]]
                     } for recipe in recipes]
    
    return randomRecipesWeighted, randomRecipesWeightedCategories, randomRecipes, randomRecipesCategories

null_models = [generateRandomRecipes(recipes, categoryRecipes, categoryComponents, ingrFreq,
                          categoryFreq, totalIngredientsUsed, ingredient_distribution[cuisine_idx][0],
                                     ingredient_distribution[cuisine_idx][1], ingredient_distribution[cuisine_idx][2])
               for cuisine_idx, (recipes, categoryRecipes, categoryComponents, ingrFreq,
                          categoryFreq, totalIngredientsUsed) in enumerate(cuisine_recipes.values())]
model_names = ["Ingredient Frequency Conserving", "Ingredient Frequency and Ingredient Category Preserving",
              "Uniform Random", "Uniform Random and Ingredient Category Preserving"]

In [ ]:
figure_title = "Ingredient Distribution Null Model of %s on %s Cuisine:"

for cuisine_index, cuisine_name in enumerate(cuisine_recipes.keys()):
    for model_index, model in enumerate(null_models[cuisine_index]):
        numIngredients(figure_title % (model_names[model_index],cuisine_name), model)

### Number of Shared Compounds
Determines the mean number of shared compounds, $N_s$ for a recipe $R$ that contains $n_R$ unique ingredients, such that each ingredient $i$ has a set of compounds $C_i$. Formula taken from paper is:

$$ N_s(R)=\frac{2}{n_R(n_R-1)} \sum_{i,j \in R, i \ne j} |C_i \cap C_j| $$

In [ ]:
# a dictionary of {ingredientName: listOfCompounds}
compoundMap = {}

with open(COMPOUNDS_PATH) as compoundFile:
    compoundMap = json.load(compoundFile) # WARNING! This file is 150 MB

In [ ]:
# Memoize Shared Compounds for Faster Computation
shared_compounds = dict()

# Calculates number of shared components
def nsc(ingredients, compounds):
    if len(ingredients) == 1:
        return 0
    
    numShared = 0
    for index, ingr1 in enumerate(ingredients):
        for ingr2 in ingredients[index+1:]:
            if ingr1 != ingr2 and ingr1 in compounds and ingr2 in compounds:
                if not (ingr1,ingr2) in shared_compounds:
                    shared_compounds[(ingr1,ingr2)] = len(set(compounds[ingr1]).intersection(compounds[ingr2]))
                numShared += shared_compounds[(ingr1,ingr2)]
                
    return 2*numShared / ( len(ingredients)*(len(ingredients)-1) )

Lets calculate the number of shared compounds of some pairs of ingredients:

In [ ]:
nsc(["Shrimp", "Lemon"], compoundMap)

In [ ]:
nsc(["Cattle (Beef, Veal)", "Arabica coffee"], compoundMap)

... and for a recipe:

In [ ]:
recipe = cuisine_recipes["Russian"][0][0]
print("'%s': %.2f" % (recipe["name"], nsc(recipe["ingredients"], compoundMap)) )

##### Shared compounds of randomly generated recipes
We calculate the distribution of number of shared components for random recipes and see how it compares to a real set of recipes. This can then be used to determine the significance of a cuisines number of shared compounds.

Given a cuisine $c$ with $N_c$ recipes, the mean number of shared compoounds $N_s$ is given by

$$ N_s = \sum_R \frac{N_s(R)}{N_c} $$

We can use this to measure the significance of a cuisines mean number of shared compounds through the metric $\Delta N_s = N_{s}^{c} - N_{s}^{rand}$. This metric can also be used to calculate a Z-score and determine statistical significance.

In [ ]:
def getNumSharedCompounds(recipes, title):
    NsScores = [nsc(recipe["ingredients"], compoundMap) for recipe in recipes]

    plt.title(title)
    plt.xlabel("Number of Shared Compounds")
    plt.ylabel("Occurances")
    plt.hist(NsScores, bins=50, color="#2e8b57")
    plt.vlines(stats.tmean(NsScores), 0, plt.gca().get_ylim()[1], colors="r", label="Mean")
    plt.legend()
    plt.show()

    print("mean={}".format(stats.tmean(NsScores)))
    print("stddev={}".format(stats.tstd(NsScores)))
    print("range=({}, {})".format(min(NsScores), max(NsScores)))

    Ns = stats.tmean(NsScores)
    return Ns, NsScores

NsRandAndScores = [[getNumSharedCompounds(null_model, cuisine_name + " " + model_names[index] + " Random Recipes")
                   for index, null_model in enumerate(null_models[cuisine_index])]
                for cuisine_index, cuisine_name in enumerate(cuisine_recipes.keys())]

##### Shared Compounds of recipe dataset

In [ ]:
NsCuisines = {cuisine_name: getNumSharedCompounds(data[0], cuisine_name + " $N_s$")
              for cuisine_name, data in cuisine_recipes.items()}

##### The Stats
It is now possible to calculate the marginal number of shared components for the russian cuisine.

Assuming random recipes are normally distributed* it is then also possible to calculate a Z-score for the cuisine.

*See report for discussion or errors and justification

In [ ]:
def getZScore(cuisineName, NsCuisine, null_model_name, NsRandAndScores):
    # Get Z-Score
    NsRand, NsRandScores = NsRandAndScores
    deltaNsCuisine = NsCuisine - NsRand
    zCuisine = deltaNsCuisine / stats.tstd(NsRandScores)

    print("deltaNs(%s on %s)=%f" % (cuisine_name, null_model_name + " Model",
                                    deltaNsCuisine))
    print(zCuisine)
    print("Z-score=%f" % (zCuisine))

    #show graphically
    NsRandScores = NsRandAndScores[1]
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.title("Distributions of $N_s$ for %s Random Recipes" % (null_model_name))
    plt.xlabel("Number of Shared Compounds, $N_s$")
    plt.ylabel("P($N_s$)")
    plt.hist(NsRandScores, bins=50, weights=(1/len(NsRandScores))*np.ones_like(NsRandScores), color="#2e8b57")
    ylim = plt.gca().get_ylim()[1]
    plt.vlines(stats.tmean(NsRandScores), 0, ylim, colors="r", label="Random Recipe Mean")
    plt.vlines(NsCuisine, 0, ylim, colors="b", label=cuisineName)
    plt.legend()
    plt.show()
    
    return zCuisine
    
zScores = [[getZScore(cuisine_name, NsCuisines[cuisine_name][0], \
            model_names[model_index], NsRandAndScores[cuisine_index][model_index])
  for model_index, model in enumerate(null_models[cuisine_index])]
 for cuisine_index, cuisine_name in enumerate(cuisine_recipes.keys())]

In [ ]:
random.seed("seeeeeeed")

def randomColour():
    return "#"  + "".join([random.choice("0123456789ABCDEF") for halfbyte in range(6)])

## Plot Z-Scores Groups By Null Model
cuisine_name_list = world_cuisine_list + ["Russian"]
cuisine_colour_list = [randomColour() for cuisine in cuisine_name_list]
null_model_zscores = list(zip(*zScores)) # Group Z-Scores by Null Model
for model_index, cuisine_zscores in enumerate(null_model_zscores):
    #show graphically
    plt.figure(figsize=(12, 4))
    plt.title("Z-Scores on %s Null Model by Cuisine" % (model_names[model_index]))
    plt.xlabel("Std. Devs. from Mean")
    plt.ylabel("Z")

    xx = np.linspace(-4, 4, 100)
    plt.plot(xx, stats.norm.pdf(xx, loc=0, scale=1), color="k", lw=2)

    ylim = plt.gca().get_ylim()[1]
    plt.vlines(0, 0, stats.norm.pdf(0, loc=0, scale=1), linestyles="dotted", colors="#aaaaaa")
    for cuisine_index, zCuisine in enumerate(cuisine_zscores):
        plt.vlines(zCuisine, 0, ylim, colors=cuisine_colour_list[cuisine_index], \
                   label=cuisine_name_list[cuisine_index])
    plt.legend()
    plt.show()

### Authenticity
The authenticity of an ingredient $i$ in a cuisine $c$ can be determined from its prevalence $P_i^c$, where $P_i^c$ is the proportion of recipes from $c$ which contain $i$:

$$P_i^c = \frac{\textrm{no. recipes in }c\textrm{ containing }i}{\textrm{no. recipes in }c} = \frac{n_i^c}{N_i^c} $$

The authenticity $p_i^c$ is defined as the difference between the prevalence for c and the average prevalence for all other cuisines:

$$ p_i^c = P_i^c - \left\langle P_i^{c'} \right\rangle_{c' \neq c}$$

The authenticity of ingredient pairs $p_{ij}^c$ and triplets $p_{ijk}^c$ can be determined similarly.

In [ ]:
# Calculates the mean of an interable. Empty collections are deemed to have an average of 0.
def mean(array):
    if len(array) == 0:
        return 0
    return sum(array) / len(array)

In [ ]:
#Calculate the prevalence of a cuisine
def prevalence(cuisine_recipes, size):
    prevalence_cuisine = dict()
    for cuisine_name, data in cuisine_recipes.items():
        # Get number of recipes containing some ingredient combination
        if cuisine_name not in prevalence_cuisine:
            prevalence_cuisine[cuisine_name] = dict()
        for recipe in data[0]:
            if len(recipe["ingredients"]) >= size:
                combinations = itertools.combinations(recipe["ingredients"], size)
                for combination in combinations:
                    if tuple(sorted(combination)) not in prevalence_cuisine[cuisine_name]:
                        prevalence_cuisine[cuisine_name][tuple(sorted(combination))] = 0
                    prevalence_cuisine[cuisine_name][tuple(sorted(combination))] += 1
        # Divide by number of recipes
        prevalence_cuisine[cuisine_name] = {combination: prevalence_cuisine[cuisine_name][combination] / len(data[0])
                                            for combination in prevalence_cuisine[cuisine_name]}
    return prevalence_cuisine

In [ ]:
# returns the the authenticity of a groups of ingredients with size 'size'
def authenticity(cuisine_recipes, size):
    prevalence_cuisine = prevalence(cuisine_recipes, size)
    authenticity_cuisine = {cuisine_name: {combination: prevalence_cuisine[cuisine_name][combination]
                                          - mean([prevalence_cuisine[other_cuisine_name][combination]
                                          for other_cuisine_name in cuisine_recipes.keys()
                                              if other_cuisine_name != cuisine_name
                                              and combination in prevalence_cuisine[other_cuisine_name]])
                                          for combination in prevalence_cuisine[cuisine_name]}
                                          for cuisine_name in cuisine_recipes.keys()}
    return authenticity_cuisine

Lets put all the above to the test and calculate the most authentic groups ingredients for each cuisine. We will find the most authentic ingredients, ingredient pairs, ingredient triples and even ingredient quadruples *(which was not featured in the original paper)*

In [ ]:
for size in range(4):
    print("Finding Most Authentic Combination of %d Ingredients Per Cuisine:" % (size+1))
    authenticity_cuisine = authenticity(cuisine_recipes, size+1)
    for cuisine_name in cuisine_recipes.keys():
        print("%s: " % (cuisine_name))
        print(sorted(authenticity_cuisine[cuisine_name], key=lambda combin: authenticity_cuisine[cuisine_name][combin], reverse=True)[:6])
        print()
    print()